<a href="https://colab.research.google.com/github/hchaparov/Dynamic_Pricing_MARL/blob/main/Bachelor_thesis_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment

**Stationary Demand market:**



In [2]:
!pip install gymnasium
!pip install numpy
!pip install torch
!pip install stable_baselines3 #"stable-baselines3[extra]>=2.0.0a4"
!pip install huggingface_sb3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [20]:
from pickle import TRUE
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

class OligopolyMarketEnv(gym.Env):

   # Because of google colab, we cannot implement the GUI ('human' render mode)
    metadata = {"render_modes": ["console"]}

    def __init__(self, a, b, beta_G, beta_L, reference_price, c, a_phi, before, max_steps, render_mode="console"):
        super(OligopolyMarketEnv, self).__init__()
        self.render_mode = render_mode
        self.reference_price = reference_price
        self.a = a
        self.b = b
        self.beta_G = beta_G
        self.beta_L = beta_L
        self.c = c  # costs (Lower boundary for prices)
        self.a_phi = a_phi
        self.before = before
        self.max_steps = max_steps # change of the price |max_steps| per season
        self.last_action = None
        self.last_profit = None
        self.uv_buffer = []
        self.revenue_buffer = []
        self.t = 0
        self.upper_bound = None
        action = 0
        self.count = max_steps



         # Determine the upper bound of the action space
        if self.reference_price <= self.a / self.b:
            self.upper_bound = min((self.a + self.beta_G * self.reference_price) / (self.b + self.beta_G), (self.a - self.b * self.reference_price + self.beta_G * self.reference_price)/(self.beta_G))
        else:
            self.upper_bound = min((self.a + self.beta_L * self.reference_price) / (self.b + self.beta_L), (self.a - self.b * self.reference_price + self.beta_L * self.reference_price)/(self.beta_L))

        # Ensure the upper bound is at least greater than the lower bound c
        if self.upper_bound < self.c:
           print("Watch out: upper_bound < costs")

        # Action space (price set by the firm) is bounded
        # self.action_space = spaces.Box(low=np.array([c], dtype=np.float32), high=np.array([upper_bound], dtype=np.float32), dtype=np.float32)
        # Normalize action [-1, 1]
        self.action_space = spaces.Box(low=np.array([-1], dtype=np.float32), high=np.array([1], dtype=np.float32), dtype=np.float32)

        # State space is the reference price
        self.observation_space = spaces.Box(low=np.array([0], dtype=np.float32), high=np.array([np.inf], dtype=np.float32), shape=(1,), dtype=np.float32)

    def step(self, action):
        # rescale the action space [c, upper_bound]
        # action = action
        rescaled_action = self.c + ((action + 1.01) / 2 )* (self.upper_bound - self.c)
        price = rescaled_action
        done = False
        # average_price = price  # Simplified for single agent
        # b_phi = 1


        # Implementing demand function
        if self.reference_price > price:
            demand = self.a - self.b * self.reference_price + self.beta_G * (self.reference_price - price)
        else:
            demand = self.a - self.b * self.reference_price + self.beta_L * (self.reference_price - price)

        demand = np.array(demand).item()
        demand = int(np.floor(demand))

        # Ensuring float type inside the environment
        # Converting from array to scalar if needed
        if isinstance(price, np.ndarray):
          price = price.item()

        if isinstance(demand, np.ndarray):
          demand = demand.item()

        revenue = price * demand
        self.revenue_buffer.append(revenue)
        self.uv_buffer.append(np.random.randint(demand + 1, 1000))

        # immediate reward function
        if self.t == 0 or self.before >= self.t:
           reward = self.revenue_buffer[self.t] / self.uv_buffer[self.t]
        else:
           reward = (self.revenue_buffer[self.t] / self.uv_buffer[self.t]) - (self.revenue_buffer[self.t - self.before] / self.uv_buffer[self.t - self.before])


        # Store the last action and last profit for render()
        self.last_action = rescaled_action
        self.last_profit = revenue

        next_state = np.array([self.reference_price]).astype(np.float32)  # State is constant
        self.t += 1

        if self.t == self.max_steps:
          done = True
          self.max_steps += self.count


        return next_state, reward, done, False, {} #, action, price, demand, self.t, rescaled_action, self.revenue_buffer, self.uv_buffer


    def reset(self, seed=None, options=None):
        super().reset(seed=seed, options=options)
       # we convert to float32 to make it more general, because we want to use continuous actions
        return np.array([self.reference_price]).astype(np.float32), {}


    def render(self):
      if self.render_mode == "console":
          if self.last_action is not None and self.last_profit is not None:
              # Ensure last_action and last_profit are scalars for formatting
              if isinstance(self.last_action, np.ndarray):
                  if self.last_action.size == 1:
                      last_action = self.last_action.item()  # Convert single element array to scalar
                  else:
                      last_action = self.last_action
                      print("Error: last_action is not a single element array")
              else:
                  last_action = self.last_action  # if already scalar

              if isinstance(self.last_profit, np.ndarray):
                  if self.last_profit.size == 1:
                      last_profit = self.last_profit.item()
                  else:
                      last_profit = self.last_profit
                      print("Error: last_profit is not a single element array")
              else:
                  last_profit = self.last_profit

              print(f"Reference Price: {self.reference_price:.3f}")
              print(f"Last Action (Price Set by Firm): {last_action:.3f}")
              print(f"Last Profit: {last_profit:.3f}")


    def close(self):
        pass



TEST OF THE NON-TRAINED ENV:

In [4]:
#Test for compatability with the algorithms of stable_baselines3
from stable_baselines3.common.env_checker import check_env
env = OligopolyMarketEnv(a = 10, b = 1, beta_G = 2, beta_L = 2, reference_price = 1.0, c = 0, a_phi = 1, before = 2, max_steps = 3)
# If the environment doesn't follow the interface, an error will be thrown
check_env(env, warn=True)

In [12]:
import random
env = OligopolyMarketEnv(a = 10, b = 1, beta_G = 2, beta_L = 2, reference_price = 1.0, c = 0, a_phi = 1, before = 2, max_steps = 3)

obs, _ = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())


# Test:
n_steps = 20
for step in range(n_steps):
    price = random.uniform(-1, 1)
    print(f"Step {step + 1}")
    obs, reward, done, false, info, action, price, demand, t, rescaled_action, revenue_buffer, uv_buffer = env.step(price)
    print( "reward=", reward, "action=", action, "done =", done, ",price=", price, ",demand=", demand,",t = ", t, ",rescaled_action=", rescaled_action, ",revenue_buffer=", revenue_buffer, ",uv_buffer=", uv_buffer)
    env.render()

Box(0.0, inf, (1,), float32)
Box(-1.0, 1.0, (1,), float32)
[-0.50006974]
Step 1
reward= 0.0008989268667747776 action= -0.9683347397249891 done = False ,price= 0.08333052055002188 ,demand= 10 ,t =  1 ,rescaled_action= 0.08333052055002188 ,revenue_buffer= [0.8333052055002188] ,uv_buffer= [927]
Reference Price: 1.000
Last Action (Price Set by Firm): 0.083
Last Profit: 0.833
Step 2
reward= 0.06958028688963223 action= 0.7208096363796019 done = False ,price= 3.461619272759204 ,demand= 4 ,t =  2 ,rescaled_action= 3.461619272759204 ,revenue_buffer= [0.8333052055002188, 13.846477091036816] ,uv_buffer= [927, 199]
Reference Price: 1.000
Last Action (Price Set by Firm): 3.462
Last Profit: 13.846
Step 3
reward= 0.015302518936511543 action= 0.5145134490499625 done = True ,price= 3.049026898099925 ,demand= 4 ,t =  3 ,rescaled_action= 3.049026898099925 ,revenue_buffer= [0.8333052055002188, 13.846477091036816, 12.1961075923997] ,uv_buffer= [927, 199, 797]
Reference Price: 1.000
Last Action (Price Set b

Vectorize the environment:

In [5]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor

vec_env = make_vec_env(OligopolyMarketEnv, n_envs=10, env_kwargs=dict(a = 10, b = 1, beta_G = 2, beta_L = 2, reference_price = 1.0, c = 0, a_phi = 1, before = 2, max_steps = 3))

# **PPO model**

In [28]:
model = PPO(
    policy = 'MlpPolicy',
    env = vec_env,
    seed = 0,
    n_steps = 1970,
    batch_size = 128,
    n_epochs = 5,
    gamma = 0.9601882016991302,
    gae_lambda = 0.889617767476432,
    ent_coef = 0.027907021786249108,
    verbose=1).learn(10000000)

model_name = "ppo-Oligopoly"
model.save(model_name)


Streaming output truncated to the last 5000 lines.
|    loss                 | 0.0108       |
|    n_updates            | 1400         |
|    policy_gradient_loss | -0.000151    |
|    std                  | 0.354        |
|    value_loss           | 0.0788       |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3            |
|    ep_rew_mean          | 0.000569     |
| time/                   |              |
|    fps                  | 3007         |
|    iterations           | 282          |
|    time_elapsed         | 1847         |
|    total_timesteps      | 5555400      |
| train/                  |              |
|    approx_kl            | 0.0002661744 |
|    clip_fraction        | 0.00141      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.387       |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|  

# Automatic Hyperparameter Tuning:

In [7]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 9.4 MB/s eta 0:00:00


In [26]:
import optuna

def objective(trial):

    # Hyperparameter boundaries
    n_steps = trial.suggest_int('n_steps', 256, 2048)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    n_epochs = trial.suggest_int('n_epochs', 3, 10)
    gamma = trial.suggest_uniform('gamma', 0.9, 0.9999)
    gae_lambda = trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    ent_coef = trial.suggest_loguniform('ent_coef', 0.01, 0.1)

    # Setup the model
    model = PPO(
        policy='MlpPolicy',
        env=vec_env,
        seed=0,
        n_steps=n_steps,
        batch_size=batch_size,
        n_epochs=n_epochs,
        gamma=gamma,
        gae_lambda=gae_lambda,
        ent_coef=ent_coef,
        verbose=1
    )

    # Train the model
    model.learn(total_timesteps=20000)

    # Evaluate the model
    obs = vec_env.reset()
    total_rewards = 0
    done = np.array([False])
    while not done.all():
        action, _ = model.predict(obs, deterministic=False)
        obs, rewards, done, _info = vec_env.step(action)
        total_rewards += rewards.sum()

    # Save the model
    model_name = f"ppo-Oligopoly-{trial.number}"
    model.save(model_name)

    return total_rewards



In [27]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Best hyperparameters
print("Best trial:", study.best_trial.params)


[I 2024-05-01 09:31:35,283] A new study created in memory with name: no-name-2f7838cd-d144-40fe-87ce-ef0053a0a940
<ipython-input-26-8c5bb3a3c566>:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  gamma = trial.suggest_uniform('gamma', 0.9, 0.9999)
<ipython-input-26-8c5bb3a3c566>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  gae_lambda = trial.suggest_uniform('gae_lambda', 0.8, 0.99)
<ipython-input-26-8c5bb3a3c566>:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  ent_coef = trial.suggest_loguniform('ent_coef', 0.01, 0.1)
/usr/local/lib/python3.

Using cpu device
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 3         |
|    ep_rew_mean     | -0.000787 |
| time/              |           |
|    fps             | 6506      |
|    iterations      | 1         |
|    time_elapsed    | 3         |
|    total_timesteps | 20190     |
----------------------------------


[I 2024-05-01 09:31:44,282] Trial 0 finished with value: 0.6998485624790192 and parameters: {'n_steps': 2019, 'batch_size': 128, 'n_epochs': 7, 'gamma': 0.9825714383907151, 'gae_lambda': 0.9425304423608996, 'ent_coef': 0.01054622780534942}. Best is trial 0 with value: 0.6998485624790192.
/usr/local/lib/python3.10/dist-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 32, but because the `RolloutBuffer` is of size `n_steps * n_envs = 19740`, after every 616 untruncated mini-batches, there will be a truncated mini-batch of size 28
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=1974 and n_envs=10)
  warnings.warn(


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3        |
|    ep_rew_mean     | 0.000178 |
| time/              |          |
|    fps             | 7737     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 19740    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3            |
|    ep_rew_mean          | 0.00197      |
| time/                   |              |
|    fps                  | 2450         |
|    iterations           | 2            |
|    time_elapsed         | 16           |
|    total_timesteps      | 39480        |
| train/                  |              |
|    approx_kl            | 0.0074380543 |
|    clip_fraction        | 0.103        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.41        |
|    explained_variance   | 0            

[I 2024-05-01 09:32:11,077] Trial 1 finished with value: 0.04398512840270996 and parameters: {'n_steps': 1974, 'batch_size': 32, 'n_epochs': 4, 'gamma': 0.997299503664903, 'gae_lambda': 0.8221147114798086, 'ent_coef': 0.0805656874655687}. Best is trial 0 with value: 0.6998485624790192.
/usr/local/lib/python3.10/dist-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 128, but because the `RolloutBuffer` is of size `n_steps * n_envs = 14810`, after every 115 untruncated mini-batches, there will be a truncated mini-batch of size 90
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=1481 and n_envs=10)
  warnings.warn(


...

Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3        |
|    ep_rew_mean     | -0.0134  |
| time/              |          |
|    fps             | 6805     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 18380    |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3           |
|    ep_rew_mean          | -0.00355    |
| time/                   |             |
|    fps                  | 4152        |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 36760       |
| train/                  |             |
|    approx_kl            | 0.011536317 |
|    clip_fraction        | 0.114       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.4        |
|    explained_variance   | 0           |
|    learning

[I 2024-05-01 10:03:41,907] Trial 98 finished with value: 0.27826379239559174 and parameters: {'n_steps': 1838, 'batch_size': 128, 'n_epochs': 4, 'gamma': 0.9738723796115645, 'gae_lambda': 0.8971925743108463, 'ent_coef': 0.030394394506007838}. Best is trial 85 with value: 3.2475749254226685.
/usr/local/lib/python3.10/dist-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 128, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5420`, after every 42 untruncated mini-batches, there will be a truncated mini-batch of size 44
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=542 and n_envs=10)
  warnings.warn(


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3        |
|    ep_rew_mean     | -0.0039  |
| time/              |          |
|    fps             | 5074     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 5420     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 3            |
|    ep_rew_mean          | -0.00528     |
| time/                   |              |
|    fps                  | 3384         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 10840        |
| train/                  |              |
|    approx_kl            | 0.0082104625 |
|    clip_fraction        | 0.0853       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.41        |
|    explained_variance   | 0            

[I 2024-05-01 10:03:50,165] Trial 99 finished with value: 0.10180149227380753 and parameters: {'n_steps': 542, 'batch_size': 128, 'n_epochs': 6, 'gamma': 0.95427976992766, 'gae_lambda': 0.8301253401560476, 'ent_coef': 0.013515682479699599}. Best is trial 85 with value: 3.2475749254226685.


Best trial: {'n_steps': 1970, 'batch_size': 128, 'n_epochs': 5, 'gamma': 0.9601882016991302, 'gae_lambda': 0.889617767476432, 'ent_coef': 0.027907021786249108}


# Evaluate policy:

In [30]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, vec_env, n_eval_episodes=10, deterministic=False)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-0.13 +/- 0.27018839343976264


# Test the agent:

In [31]:
# Test the trained agent
# using the vec_env
obs = vec_env.reset()
n_steps = 20
for step in range(n_steps):
    action, _ = model.predict(obs, deterministic = False)
    print(f"Step {step + 1}")
    print("Action: ", action)
    obs, reward, done, info = vec_env.step(action)
    print("obs=", obs, "reward=", reward, "done=", done)
    env.render()
    if done.all == True:
      print("End of the year", "reward=", reward)


Step 1
Action:  [[0.37647554]
 [0.38586572]
 [0.39531833]
 [0.5670901 ]
 [0.6664914 ]
 [1.        ]
 [0.03694803]
 [0.54122907]
 [0.61086214]
 [0.4967507 ]]
obs= [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]] reward= [-0.01097453  0.02483624  0.27102295 -0.01427257 -0.02824794  0.01761931
  0.00595464 -0.00077885  0.06957282  0.01603241] done= [False False False False False False False False False False]
Reference Price: 1.000
Last Action (Price Set by Firm): 0.292
Last Profit: 2.916
Step 2
Action:  [[ 0.9534478 ]
 [ 1.        ]
 [ 0.65922356]
 [-0.34054   ]
 [ 0.84296197]
 [ 0.66949594]
 [ 0.42006978]
 [ 0.6019601 ]
 [-0.5994081 ]
 [ 1.        ]]
obs= [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]] reward= [-0.00535809  0.00842392 -0.26195234 -0.0133842  -0.00658159  0.22111201
  0.15131332 -0.00393229  0.04154103  0.00313133] done= [False False False False False False False False False False]
Reference Price: 1.000
Last Action (Price Set by Firm): 0.292
Las